In [ ]:
# # #  Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Attention, Add, Reshape, RepeatVector, Concatenate, Permute

from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense, Layer, MultiHeadAttention, Add, LayerNormalization, Dropout

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # #  import data
df = pd.read_csv('/content/drive/MyDrive/labtest/forestfires.csv')

In [ ]:
# # #  Parameter configurations
SIGMA = 0.5
PERCENTAGE_OUTLIER = 6*8
OUTLIER_PECENTAGE = 5
PERCENTAGE_REPAIRED = 5*8

length = df.shape[0]

columns = df.columns.tolist()


In [ ]:
# # # Preprocessing data
def day_name_to_num(day_name):
    day_list = {'mon':1, 'tue':2, 'wed':3, 'thu':4, 'fri':5, 'sat':6, 'sun':7}
    for item, num in day_list.items():
        if day_name == item:
            return(num)

df['month'] = pd.to_datetime(df.month, format='%b').dt.month

# Convert day to numerial
for i in range(length):
    df.loc[i,'day'] = day_name_to_num(df.loc[i, 'day'])

# Label the outlier and inlier
for i in range(length):
    df.loc[i, 'index'] = i
    df.loc[i, 'label'] = 1

# Select random index for outlier
random_indices = np.random.choice(df.index, PERCENTAGE_OUTLIER, replace = False)

# Creating outlier data (temperature or humidity)
for index in random_indices:
    df_1 = df.iloc[index]
    temp = df_1['RH']

    sigma_T = SIGMA * temp
    noise_T = random.gauss(0, sigma_T)

    temp += noise_T
    df_1['RH']  = temp
    df_1['label'] = 0

    length +=1
    df.loc[length] = df_1
    # df.loc[index] = df_1

# Shuffling the dataset
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# # # # FID for repairing data
# def FID_repaired(working_list):
#     # working_list = x1

#     index = []
#     for i, j in enumerate(working_list):
#         if j == 'NaN':
#             index.append(i)

#     # count the number of NaN/compromised point
#     p1 = working_list.count('NaN')
#     # print(p1)

#     t = p1 # Total number of missing value

#     # Select the min & max from list
#     # working_list.remove('NaN')
#     count=0
#     for index_pos in index:
#         working_list.pop(index_pos-count)
#         count+=1


#     # find mean of all observed values
#     mean = np.mean(working_list)


#     #find min value
#     a = min(working_list)

#     #find max value
#     b = max(working_list)

#     # Calculate h = (b-a)/t
#     h = (b-a)/t

#     # Calculate the discrete universe U using u = (a + (s-1) x h + a + s x h)/2, s=1,2,3
#     U = []
#     for s in range(1,t+1):
#         u = (a + (s-1) * h + a + s * h)/2
#         U.append(u)

#     # print(U)

#     # Calculating the missing values
#     M = []
#     for u in U:
#         # print(U)

#         # Compute the contribution weight (micro) of each observed element x_i

#         contribution_weight_list = []

#         for i in working_list:
#             if abs(i-u) <= h:
#                 temp = 1-(abs(i-u)/h)
#             else:
#                 temp = 0
#             contribution_weight_list.append(temp)

#         # Calculate the sum of x_i to u1:
#         sum_contribution_weight_list = sum(contribution_weight_list)
#         # print(sum_contribution_weight_list)

#         # Calculate the contribution of an observed data x_i
#         sum_contribution_observed_data = []

#         for num1, num2 in zip(working_list, contribution_weight_list):
#         	sum_contribution_observed_data.append(num1 * num2)

#         sum_contribution_observed_data = sum(sum_contribution_observed_data)
#         # print(sum_contribution_observed_data)

#         # Calculate the missing values in x_i
#         if sum_contribution_weight_list == 0:
#             m = mean
#         else:
#             m = sum_contribution_observed_data/sum_contribution_weight_list

#         M.append(m)

#     # print('The values:',M)
#     # print('The index position:',index)
#     return [index,M]

In [ ]:
# # # # Modified the dataset
# data = df['RH'].to_list()

# # # # Determine the outlier for repairing
# index_outlier = []

# y = df['label']
# count_outlier = 0
# for i, j in enumerate(y):
#     if j == 0.0:
#         index_outlier.append(i)
#         count_outlier+=1
#     if count_outlier == PERCENTAGE_REPAIRED: break

# # Determine the compromised data
# for ind in index_outlier:
#     data[ind] = 'NaN'

# # Recover compromised data
# working_list = data

# results = FID_repaired(working_list)

# # print('The recovered values:',results[1])
# # print('The index position:',results[0])


In [ ]:
# # # # Update the predicted data into dataset
# pos = 0
# for index_pos in results[0]:
#   df_1 = df.iloc[index_pos]
#   df_1['RH'] = results[1][pos]
#   df_1['label'] = 1

#   df.loc[index_pos] = df_1
#   pos+=1

In [ ]:
# Paper 1:

# Implementation of AMSA-VAE method for missing data filling
import numpy as np


# Standard Scaling
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df[['RH']])

# Imputation
imputer = KNNImputer(n_neighbors=5)
data_imputed = imputer.fit_transform(data_scaled)

# AMSA-VAE Model Definition
input_layer = Input(shape=(1,))
dense_1 = Dense(128, activation='relu')(input_layer)
repeat_vector = RepeatVector(1)(dense_1)
permute_layer = Permute((2, 1))(repeat_vector)
attention_output = Attention()([permute_layer, permute_layer])
flatten_attention = Reshape((128,))(attention_output)
add_layer = Add()([dense_1, flatten_attention])
output_layer = Dense(1)(add_layer)

# Compile and Train the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')
model.fit(data_imputed, data_imputed, epochs=10, batch_size=16)

# Predict and Fill Missing Data
data_filled = model.predict(data_imputed)
data_filled = scaler.inverse_transform(data_filled)
df['RH'] = data_filled.flatten()  # Update the DataFrame with repaired data


Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 1.1409
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3792
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0580
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0224
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0191
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0164
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0151
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0101
Epoch 9/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0086
Epoch 10/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0080
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [ ]:
# # Paper 2:

# # # # Applying Switching Triple-Weight-SMOTE (NSS)
# # Step 1: Impute missing values using LNMF
# nmf_model = NMF(n_components=5, init='random', random_state=0)
# W = nmf_model.fit_transform(np.nan_to_num(df[['RH']]))
# H = nmf_model.components_
# df['RH'] = np.dot(W, H).flatten()  # Impute missing values and update the DataFrame

# # Step 2: Map to Empirical Feature Space (EFS) - simplified for demonstration
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(df[['RH']])
# kmeans = KMeans(n_clusters=2, random_state=0).fit(data_scaled)
# df['cluster'] = kmeans.labels_

# # Step 3: Apply Switching Triple-Weight-SMOTE
# # Assign synthetic samples according to cluster properties
# for cluster_label in df['cluster'].unique():
#     cluster_data = df[df['cluster'] == cluster_label]
#     cluster_center = cluster_data[['RH']].mean().values
#     distances, indices = pairwise_distances_argmin_min(cluster_data[['RH']], [cluster_center])
#     # Synthesize new samples if needed based on cluster distribution
#     if cluster_label == 0:  # Example condition
#         new_samples = cluster_data.sample(frac=0.1, replace=True)
#         df = pd.concat([df, new_samples])

# # Shuffle the final DataFrame
# df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# # Paper 3:

# # AMSA-VAE implementation for data filling
# class AMSA(Layer):
#     def __init__(self, d_model, num_heads):
#         super(AMSA, self).__init__()
#         self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
#         self.norm = LayerNormalization(epsilon=1e-6)
#         self.dropout = Dropout(0.1)

#     def call(self, x):
#         # Expand dims to create the necessary 3D input (batch, seq_len, features)
#         x = tf.expand_dims(x, axis=1)  # Expands to (batch, 1, features)
#         attn_output = self.mha(x, x)  # Apply multi-head attention
#         attn_output = self.dropout(attn_output)
#         out1 = self.norm(x + attn_output)  # Add & normalize
#         out1 = tf.squeeze(out1, axis=1)  # Squeeze back to 2D if necessary
#         return out1

# def create_amsa_vae(input_shape):
#     inputs = Input(shape=input_shape)
#     x = AMSA(d_model=128, num_heads=4)(inputs)
#     x = Dense(64, activation='relu')(x)
#     outputs = Dense(input_shape[0], activation='linear')(x)
#     model = Model(inputs, outputs)
#     return model

# # Data scaling
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(df[['RH']])

# # AMSA-VAE model setup and training
# amsa_vae = create_amsa_vae((1,))
# amsa_vae.compile(optimizer='adam', loss='mse')
# amsa_vae.fit(data_scaled, data_scaled, epochs=10, batch_size=4)

# # Predict and update filled data
# data_filled = amsa_vae.predict(data_scaled)
# df['RH'] = scaler.inverse_transform(data_filled).flatten()  # Update the DataFrame with repaired data

In [ ]:
# # # Splitting the dataset
y = df['label']
X = df.drop(['index', 'label'], axis =1)

# print(np.count_nonzero(y == 0))

# Spliting data
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)


In [ ]:
# # # # Random Forest
# forest = RandomForestClassifier(n_estimators = 10, random_state = 0)
# forest.fit(x_train, y_train)
# preds = forest.predict(x_test)


# # # Metrics
# print('accuracy_score: ', round((accuracy_score(y_test,preds)*100),2))
# print('f1_score: ', round(f1_score(y_test,preds),6))
# print('precision_score: ', round(precision_score(y_test,preds),6))
# print('recall_score: ', round(recall_score(y_test,preds),6))
# tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
# specificity = tn/(tn+fp)
# print('Specificity : ', round(specificity,6))

In [ ]:
# # # # # Autoencoder

# x_train = np.asarray(x_train).astype('float32')
# x_test = np.asarray(x_test).astype('float32')
# y_train = np.asarray(y_train).astype('float32')
# y_test = np.asarray(y_test).astype('float32')

# # Splitting the for testing and validating dataset
# x_val, x_test, y_val, y_test = train_test_split(x_test,y_test, test_size=0.33,stratify=y_test)

# class AutoEncoder(Model):
#   """
#   Parameters
#   ----------
#   output_units: int
#     Number of output units

#   code_size: int
#     Number of units in bottle neck
#   """

#   def __init__(self, output_units, code_size=8):
#     super().__init__()
#     self.encoder = Sequential([
#       Dense(64, activation='relu'),
#       Dropout(0.1),
#       Dense(32, activation='relu'),
#       Dropout(0.1),
#       Dense(16, activation='relu'),
#       Dropout(0.1),
#       Dense(code_size, activation='relu')
#     ])
#     self.decoder = Sequential([
#       Dense(16, activation='relu'),
#       Dropout(0.1),
#       Dense(32, activation='relu'),
#       Dropout(0.1),
#       Dense(64, activation='relu'),
#       Dropout(0.1),
#       Dense(output_units, activation='sigmoid')
#     ])

#   def call(self, inputs):
#     encoded = self.encoder(inputs)
#     decoded = self.decoder(encoded)
#     return decoded

# model = AutoEncoder(output_units=x_train.shape[1])
# # configurations of model
# model.compile(loss='msle', metrics=['mse'], optimizer='adam')

# # simple early stopping
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

# history = model.fit(
#     x_train,
#     x_train,
#     epochs=50,
#     batch_size=128,
#     validation_data=(x_val, x_val),
#     verbose=0,
#     callbacks=[es]
# )

# def find_threshold(model, x_train_scaled):
#   # another method to find threshold
#   reconstructions = model.predict(x_train_scaled)
#   # provides losses of individual instances
#   reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)

#   threshold_2 = np.percentile(reconstruction_errors, 100-OUTLIER_PECENTAGE)
#   return threshold_2

# def get_predictions(model, x_test_scaled, threshold):
#   predictions = model.predict(x_test_scaled)
#   # provides losses of individual instances
#   errors = tf.keras.losses.msle(predictions, x_test_scaled)
#   # 0 = anomaly, 1 = normal
#   anomaly_mask = pd.Series(errors) > threshold
#   preds = anomaly_mask.map(lambda x: 0.0 if x == True else 1.0)
#   return preds

# threshold = find_threshold(model, x_train)
# preds = get_predictions(model, x_test, threshold)

# preds = np.asarray(preds).astype('float32')

# #Metrics
# print('accuracy_score: ', round((accuracy_score(y_test,preds)*100),2))
# print('f1_score: ', round(f1_score(y_test,preds),6))
# print('precision_score: ', round(precision_score(y_test,preds),6))
# print('recall_score: ', round(recall_score(y_test,preds),6))
# tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
# specificity = tn/(tn+fp)
# print('Specificity : ', round(specificity,6))

In [ ]:
# # # KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x_train, y_train)

preds = neigh.predict(x_test)

# Metrics
print('accuracy_score: ', round((accuracy_score(y_test,preds)*100),2))
print('f1_score: ', round(f1_score(y_test,preds),6))
print('precision_score: ', round(precision_score(y_test,preds),6))
print('recall_score: ', round(recall_score(y_test,preds),6))
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
specificity = tn/(tn+fp)
print('Specificity : ', round(specificity,6))

accuracy_score:  91.76
f1_score:  0.957055
precision_score:  0.917647
recall_score:  1.0
Specificity :  0.0
